In [44]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import shap

In [45]:
os.chdir('/Users/sunnyyou/Documents/Real_Time_HAI/HIECNN/IMERG')

In [46]:
train = pd.read_csv("DEV/ALL_TRAIN_DATA_RESAMPLE.csv")
train = train[["GIS_ID", "VMAX", "VMAX_N06", "VMAX_N12"]]
print(len(train.GIS_ID))
train.head()

58112


,GIS_ID,VMAX,VMAX_N06,VMAX_N12
0,ATL_200001_TD_2000060806,25,25,25
1,ATL_200001_TD_2000060812,25,25,25
2,ATL_200002_TD_2000062312,30,30,25
3,ATL_200002_TD_2000062318,30,30,30
4,ATL_200002_TD_2000062400,30,30,30


In [47]:
test = pd.read_csv("DEV/ALL_TEST_DATA.csv")
test = test[["GIS_ID", "VMAX", "VMAX_N06", "VMAX_N12"]]
print(len(test.GIS_ID))
test.head()

4312


,GIS_ID,VMAX,VMAX_N06,VMAX_N12
0,ATL_201801_TS_2018052812,50,55,55
1,ATL_201801_TS_2018052818,45,50,55
2,ATL_201801_TD_2018052900,30,45,50
3,ATL_201801_TD_2018052906,30,30,45
4,ATL_201801_TD_2018052912,30,30,30


In [48]:
a = pd.read_csv("IMERG_CSV/" + train.GIS_ID[0] + '.csv', header = None)
print(a.head())
pvmax = np.array([train.VMAX_N06[0], train.VMAX_N12[0]])
print(pvmax)

        0         1         2         3         4         5    6         7    \
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.000000   
1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.000000   
2  0.000000  0.000000  0.002614  0.035389  0.031253  0.003356  0.0  0.000000   
3  0.053374  0.086443  0.075739  0.183154  0.187500  0.022317  0.0  0.046795   
4  0.312158  0.434224  0.340077  0.413843  0.241398  0.028628  0.0  0.202615   

        8         9    ...       111       112       113  114  115  116  117  \
0  0.000000  0.000000  ...  0.327364  0.000000  0.000000  0.0  0.0  0.0  0.0   
1  0.000000  0.000000  ...  0.000000  0.000000  0.022034  0.0  0.0  0.0  0.0   
2  0.000000  0.000000  ...  0.000000  0.000000  0.022034  0.0  0.0  0.0  0.0   
3  0.354221  0.840079  ...  0.000000  0.295887  0.192012  0.0  0.0  0.0  0.0   
4  1.544581  4.629189  ...  0.100727  0.147943  0.047216  0.0  0.0  0.0  0.0   

   118  119  120  
0  0.0  0.0  0.0  


In [49]:
train_img = []
train_vmax = []
train_label = []
test_img = []
test_vmax = []
test_label = []
for f in range(len(train.GIS_ID)) :
    filename = f"IMERG_CSV/{train.GIS_ID[f]}.csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[30:91]
        temp = temp.iloc[:, 30:91]
        temp = np.array(temp)
        train_img.append(temp)
        lab = train.VMAX[f]
        train_label.append(lab)
        pvmax = np.array([train.VMAX_N06[f], train.VMAX_N12[f]])
        train_vmax.append(pvmax)
    except Exception as e:
        pass

for f in range(len(test.GIS_ID)) :
    filename = f"IMERG_CSV/{test.GIS_ID[f]}.csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[30:91]
        temp = temp.iloc[:, 30:91]
        temp = np.array(temp)
        test_img.append(temp)
        lab = test.VMAX[f]
        test_label.append(lab)
        pvmax = np.array([test.VMAX_N06[f], test.VMAX_N12[f]])
        test_vmax.append(pvmax)
    except Exception as e:
        pass

In [50]:
print(len(train_img))
print(len(train_vmax))
print(len(train_label))
print(len(test_img))
print(len(test_vmax))
print(len(test_label))

58112
58112
58112
4312
4312
4312


In [51]:
X_train_img = train_img
X_train_vmax = train_vmax
y_train = train_label
X_test_img = test_img
X_test_vmax = test_vmax
y_test = test_label

In [52]:
X_train_img = np.array(X_train_img)
X_train_img = X_train_img.reshape(-1,61,61,1)
X_train_img = X_train_img.astype('float32')
X_train_vmax = np.array(X_train_vmax)
X_train_vmax = X_train_vmax.reshape(-1,2)
y_train = np.array(y_train)

In [53]:
X_test_img = np.array(X_test_img)
X_test_img = X_test_img.reshape(-1,61,61,1)
X_test_img = X_test_img.astype('float32')
X_test_vmax = np.array(X_test_vmax)
X_test_vmax = X_test_vmax.reshape(-1,2)
y_test = np.array(y_test)

In [54]:
# 300km
vmax_input = keras.Input(shape =(2,), name = "vmax_layer")
img_input = keras.Input(shape =(61, 61, 1), name = "img_layer")

w = keras.layers.Conv2D(64,12) (img_input)
w = keras.layers.Conv2D(64,12)(w)
w = keras.layers.Conv2D(64,2)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.linear(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 9)(w)
w = keras.layers.Conv2D(64, 9)(w)
w = keras.layers.Conv2D(256,2)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, vmax_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(170, activation = 'linear')(output_layer1)

new_model1 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 12) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,2, activation = 'linear')(x)
x = keras.layers.Conv2D(128,7)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,2, activation = 'linear')(x)
x = keras.layers.Conv2D(64,4)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.linear(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, vmax_input])
output_layer2 = keras.layers.Dense(170, activation = 'linear')(merged_model2)

new_model2 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (10,4))(img_input)
y = keras.layers.Conv2D(256, (4,10))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,6)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,4)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.linear(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, vmax_input])
output_layer3 = keras.layers.Dense(170, activation = 'linear')(merged_model3)

new_model3 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

# z = keras.layers.Conv2D(256, 10)(img_input)
# z = keras.layers.MaxPool2D(2,2)(z)
# img_output4 = keras.layers.Flatten()(z)

# merged_model4 = keras.layers.concatenate([img_output4, vmax_input])
# output_layer4 = keras.layers.Dense(1, activation = 'sigmoid')(merged_model4)

# new_model4 = keras.Model(inputs = [img_input, vmax_input], outputs = output_layer4, name = "model_4")

# new_model4.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img_layer (InputLayer)      [(None, 61, 61, 1)]          0         []                            
                                                                                                  
 conv2d_15 (Conv2D)          (None, 50, 50, 64)           9280      ['img_layer[0][0]']           
                                                                                                  
 conv2d_16 (Conv2D)          (None, 39, 39, 64)           589888    ['conv2d_15[0][0]']           
                                                                                                  
 conv2d_17 (Conv2D)          (None, 38, 38, 64)           16448     ['conv2d_16[0][0]']           
                                                                                            

In [12]:
os.chdir("../")
models = {new_model1: '1', new_model2: '2', new_model3: '3'}
ep = range(10, 11)
batches = [1, 2, 4, 8, 16]
for m in models.keys():
    for e in ep:
        for b in batches:
            with tf.device('/GPU:0'):
                m.compile(optimizer = 'adam', loss = tf.keras.losses.MeanAbsoluteError(), metrics = ['mae', 'mse'])
                m.fit([X_train_img, X_train_vmax], y_train, epochs = e, batch_size = b)
                tf.keras.models.save_model(m, f"MODELS/MODEL{models[m]}_EPOCHS{e}_BATCH{b}")
                res = m.evaluate([X_test_img, X_test_vmax], y_test)
                print("MAE = " + str(res[0]))
                print("RMSE = " + str((res[2]) ** 0.5))
                preds = m.predict([X_test_img, X_test_vmax])
                x = np.average(preds, axis = 1)
                test['preds'] = x
                test.to_csv(f"OUTPUT/MODEL{models[m]}_RESULTS{e}_{b}.csv")
                tf.keras.backend.clear_session()

Epoch 1/10


2024-05-31 23:41:32.119410: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 9800/58112 [====>.........................] - ETA: 7:30 - loss: 6.4918 - mae: 6.4918 - mse: 88.8976

KeyboardInterrupt: 

In [55]:
os.chdir("/Users/sunnyyou/Documents/Real_Time_HAI/HIECNN/IMERG")

In [56]:
train_no_resample = pd.read_csv("DEV/ALL_TRAIN_DATA.csv")
train_no_resample = train_no_resample[["GIS_ID", "VMAX", "VMAX_N06", "VMAX_N12", "CAT"]]
train.head()

,GIS_ID,VMAX,VMAX_N06,VMAX_N12
0,ATL_200001_TD_2000060806,25,25,25
1,ATL_200001_TD_2000060812,25,25,25
2,ATL_200002_TD_2000062312,30,30,25
3,ATL_200002_TD_2000062318,30,30,30
4,ATL_200002_TD_2000062400,30,30,30


In [57]:
train_no_resample['CAT'].value_counts()['Maj']

3816

In [58]:
train_no_resample_TD = train_no_resample[train_no_resample['CAT'] == 'TD']
train_no_resample_TD = train_no_resample_TD.sample(750)
train_no_resample_TS = train_no_resample[train_no_resample['CAT'] == 'TS']
train_no_resample_TS = train_no_resample_TS.sample(750)
train_no_resample_Min = train_no_resample[train_no_resample['CAT'] == 'Min']
train_no_resample_Min = train_no_resample_Min.sample(750)
train_no_resample_Maj = train_no_resample[train_no_resample['CAT'] == 'Maj']
train_no_resample_Maj = train_no_resample_Maj.sample(750)
shap_train = pd.concat([train_no_resample_TD, train_no_resample_TS, train_no_resample_Min, train_no_resample_Maj])
shap_train = shap_train.reset_index(drop = True)
shap_train

,GIS_ID,VMAX,VMAX_N06,VMAX_N12,CAT
0,SIO_201321_TD_2013040412,25,25,25,TD
1,NWP_200319_TD_2003101200,30,30,30,TD
2,NIO_200404_TD_2004110400,30,25,25,TD
3,NIO_201701_TD_2017041412,25,25,25,TD
4,CPA_200008_TD_2000080306,25,25,25,TD
...,...,...,...,...,...
2995,SIO_201212_C4_2012021318,115,120,115,Maj
2996,SIO_200705_C4_2006122012,120,130,135,Maj
2997,NWP_201511_C4_2015071012,115,120,125,Maj
2998,EPA_200611_C3_2006083000,100,95,75,Maj


In [59]:
shap_train_img = []
shap_train_ships = []
shap_train_label = []
for f in range(len(shap_train.GIS_ID)) :
    filename = f"IMERG_CSV/{shap_train.GIS_ID[f]}.csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[30:91]
        temp = temp.iloc[:, 30:91]
        temp = np.array(temp)
        shap_train_img.append(temp)
        lab = shap_train.VMAX[f]
        shap_train_label.append(lab)
        pvmax = np.array([shap_train.VMAX_N06[f], shap_train.VMAX_N12[f]])
        shap_train_ships.append(pvmax)
    except Exception as e:
        pass

In [60]:
shap_train_img = np.array(shap_train_img)
shap_train_img = shap_train_img.reshape(-1,61,61,1)
shap_train_img = shap_train_img.astype('float32')
shap_train_ships = np.array(shap_train_ships)
shap_train_ships = shap_train_ships.reshape(-1,2)
shap_train_label = np.array(shap_train_label)

In [61]:
len(shap_train_label)

3000

In [63]:
new_model1.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model2.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model3.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])

In [64]:
new_model2.fit([X_train_img, X_train_vmax], y_train, epochs=1, batch_size=4, validation_split = 0.1)
res = new_model2.evaluate([X_test_img, X_test_vmax], y_test)
print("MAE = " + str(res[0]))
print("RMSE = " + str((res[2]) ** 0.5))
preds = new_model2.predict([X_test_img, X_test_vmax])
a = np.average(preds, axis = 1)
test['preds'] = a

 1516/13075 [==>...........................] - ETA: 2:00 - loss: 11.8414 - mae: 11.8414 - mse: 360.6414

In [ ]:
shap.explainers._deep.deep_tf.op_handlers["FusedBatchNormV3"] = shap.explainers._deep.deep_tf.linearity_1d(0)
e = shap.DeepExplainer(new_model2, [shap_train_img, shap_train_ships])
shap_values = e.shap_values([X_test_img, X_test_vmax])